# Parish maps browser

This notebook displays information about the SLV's collection of parish maps on a modern basemap. It's mainly intended as a way of visualising the results of my efforts to create bounding boxes from the available metadata.

Click on **Run > Run all cells** then select a parish map from the dropdown list:

- if there's a bounding box and an image identifier, the image of the parish map will be overlaid on the modern base map using the bounding box coordinates
- if there's a bounding box, but no image identifier, a rectangle will be drawn on the base map showing the dimensions of the bounding box
- if there are point coordinates, but no bounding box, a marker will be placed on the base map

If the image of the map is displayed you can use the slider to adjust the opacity.

Clicking on either the image, rectangle, or marker will display metadata about the parish map and a link to the SLV catalogue.

In [1]:
from ipyleaflet import ImageOverlay, Map, WidgetControl, Rectangle, Marker
import ipywidgets as widgets
import json
import pandas as pd
import requests

In [2]:
# Import the pre-processed data file
df = pd.read_csv("parish_maps_final.csv")

In [3]:
# Extract a list of titles to use in the dropdown selector
titles = df.loc[(df["latitude"].notnull()) & (df["longitude"].notnull())].sort_values("title")[["title", "publisher", "alma_id"]].to_dict(orient="records")
choices = [(f"{t['title'].strip('/')} / {t['publisher'] if pd.notnull(t['publisher']) else ''}", t["alma_id"]) for t in titles]

In [4]:
def get_iiif_id(manifest_url):
    """
    Extract a list of image @ids from an IIIF manifest
    """
    image_ids = []
    if manifest_url:
        try:
            response = requests.get(manifest_url, timeout=5)
        except requests.exceptions.Timeout:
            pass
        else:
            if response.ok:
                manifest = response.json()
                # There can be multiple images in a record
                # So we loop through the canvases to get each one.
                for canvas in manifest["sequences"][0]["canvases"]:
                    if canvas["images"][0]["resource"]["format"] == "image/jpeg":
                        image_ids.append(canvas["images"][0]["resource"]["service"]["@id"])
    return image_ids

def create_map(metadata, image_url):
    if pd.notnull(metadata["bbox"]):
        bbox = json.loads(metadata["bbox"])
    else:
        bbox = None
    default_layout = widgets.Layout(width="800px", height="600px")
    
    # Create the basemap and centre it on the map location
    m = Map(
        center=(metadata["latitude"], metadata["longitude"]),
        zoom=10,
        scroll_wheel_zoom=True,
        layout=default_layout,
    )

    html = f"<p style='line-height: 1.2;'><b>{metadata.title}</b><br>"
    html += f"{metadata.publisher}<br>" if pd.notnull(metadata.publisher) else ""
    html += f"{metadata.description}<br>" if pd.notnull(metadata.description) else ""
    html += f"{metadata.coordinates}<br>" if pd.notnull(metadata.coordinates) else ""
    html += f"{metadata.scale}<br>" if pd.notnull(metadata.scale) else ""
    html += f"<a target='_blank' href='https://find.slv.vic.gov.au/discovery/fulldisplay?docid=alma{metadata.alma_id}&vid=61SLV_INST:SLV'>View in catalogue</a><br>"
    html += "</p>"
        
    message = widgets.HTML(html)


    if bbox and image_url:
        # Add the map image as an overlay
        image = ImageOverlay(
            # Image url
            url=image_url,
            # Postion the image using the bounding box data
            bounds=((bbox[1], bbox[0]), (bbox[3], bbox[2])),
            interactive=True
        )
        image.popup = message
        # Add a slider to control the opacity of the image
        slider = widgets.FloatSlider(
            min=0,
            max=1,
            value=1,  # Opacity is valid in [0,1] range
            orientation="vertical",  # Vertical slider is what we want
            readout=False,  # No need to show exact value
            layout=widgets.Layout(width="2em"),
        )
        # Connect slider value to opacity property of the Image Layer
        widgets.jslink((slider, "value"), (image, "opacity"))
        m.add_control(WidgetControl(widget=slider))
        m.add_layer(image)

        rectangle = Rectangle(bounds=((bbox[1], bbox[0]), (bbox[3], bbox[2])), stroke=False, fill_opacity=0)
        rectangle.popup = message
        m.add(rectangle)
        #print(random["title"])
        #print(random["url"])
        # Set the map zoom so you can see all of the image.
        # m.fit_bounds([[bbox[1], bbox[0]], [bbox[3], bbox[2]]])
    elif bbox:
        # If there's no image link, draw a rectangle
        rectangle = Rectangle(bounds=((bbox[1], bbox[0]), (bbox[3], bbox[2])), line_cap="square", weight=2)
        rectangle.popup = message
        m.add(rectangle)
        #m.fit_bounds([[bbox[1], bbox[0]], [bbox[3], bbox[2]]])
    else:
        # If there's no bbox, add a marker
        marker = Marker(location=(metadata["latitude"], metadata["longitude"]))
        marker.popup = message
        m.add(marker)
    return m

def display_map(df, alma_id):
    metadata = df.loc[df["alma_id"] == alma_id].iloc[0]
    # Get an image url from IIIF manifest
    if metadata["image_id"]:
        manifest_url = f"https://rosetta.slv.vic.gov.au/delivery/iiif/presentation/2.1/{metadata['image_id']}/manifest.json"
        iiif_ids = get_iiif_id(manifest_url)
        try:
            image_url = f"{iiif_ids[0]}/full/max/0/default.jpg"
        except IndexError:
            image_url = None
    else:
        image_url = None
    m = create_map(metadata, image_url)
    display(m)

In [5]:
select = widgets.Dropdown(
    options=choices,
    description='Parish:',
    layout=widgets.Layout(width='80%', height='40px')
)
out = widgets.Output()

def on_select(change):
    out.clear_output()
    with out:
        display_map(df, change['new'])

select.observe(on_select, names='value')
display(select, out)

Dropdown(description='Parish:', layout=Layout(height='40px', width='80%'), options=(('Addington, County of Tal…

Output()